In [1]:
import tensorflow_datasets as tfd

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, GRU, LSTM, InputLayer
from tensorflow.keras.models import Sequential

import numpy as np

2021-11-25 15:59:43.735351: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-25 15:59:43.735388: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
(train, test), info = tfd.load(
    "imdb_reviews", 
    split=["train", "test",], 
    with_info=True,
    as_supervised=True
)

2021-11-25 15:59:46.711767: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-25 15:59:46.711801: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-25 15:59:46.711829: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (calcifer-Inspiron-7370): /proc/driver/nvidia/version does not exist
2021-11-25 15:59:46.712524: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
reviews_train = np.array([x[0].decode() for x in train.as_numpy_iterator()])
sentiment_train = np.array([x[1] for x in train.as_numpy_iterator()])

In [4]:
reviews_test = np.array([x[0].decode() for x in test.as_numpy_iterator()])
sentiment_test = np.array([x[1] for x in test.as_numpy_iterator()])

In [5]:
tokenizer = Tokenizer()
reviews = np.concatenate([reviews_train, reviews_test])
max_length = max(map(lambda s: len(s), reviews))
vocab_size = len(tokenizer.word_index) + 1

X_train_unpadded = tokenizer.texts_to_sequences(reviews_train)
X_test_unpadded = tokenizer.texts_to_sequences(reviews_test)

X_train = pad_sequences(X_train_unpadded, maxlen = max_length, padding='post')
X_test = pad_sequences(X_test_unpadded, maxlen = max_length, padding='post')

In [6]:
y_train = np.array(sentiment_train)
y_test = np.array(sentiment_test)

In [7]:
print(X_train.shape)
print(y_train.shape)

(25000, 13704)
(25000,)


In [13]:
EMBEDDING_DIM = 25

model = Sequential([
    Embedding(
        vocab_size, 
        EMBEDDING_DIM, 
        input_length=max_length,
        trainable=False
    ),
    GRU(units=4, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation="sigmoid"),
])

In [15]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 13704, 25)         25        
                                                                 
 gru_2 (GRU)                 (None, 4)                 372       
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 402
Trainable params: 377
Non-trainable params: 25
_________________________________________________________________


In [16]:
model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=16,
    batch_size=64,
    verbose=1
)

2021-11-25 16:04:47.585182: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1370400000 exceeds 10% of free system memory.


Epoch 1/16


KeyboardInterrupt: 